In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import warnings 
warnings.filterwarnings('ignore')

pd.set_option('max.column', None)

import plotly.express as px

In [56]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# import StandardScaler to perform scaling
from sklearn.preprocessing import StandardScaler 

# import various functions from sklearn
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV

# import the functions for visualizing the decision tree
# import pydotplus
# from IPython.display import Image  

In [6]:
# read dataset

df = pd.read_csv('Lending Club 2016_2018.csv')

In [28]:
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,tot_coll_amt,tot_cur_bal,issue_d,chargeoff_within_12_mths,acc_now_delinq,mort_acc,mths_since_recent_bc_dlq,num_accts_ever_120_pd,pct_tl_nvr_dlq,pub_rec_bankruptcies,tot_hi_cred_lim,loan_status
0,130956066,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,9 years,RENT,52000.0,Source Verified,major_purchase,WA,0.58,0.0,0.0,26.0,NaN,7.0,0.0,141.0,0.5,30.0,w,0.0,NaN,0.0,150592.0,Mar-2018,0.0,0.0,4.0,NaN,0.0,96.7,0.0,191216.0,0
1,130968727,5000.0,5000.0,5000.0,36 months,11.98,166.03,B,10+ years,OWN,55000.0,Not Verified,other,GA,14.18,0.0,0.0,74.0,82.0,14.0,1.0,11449.0,33.9,24.0,w,0.0,74.0,0.0,28880.0,Mar-2018,0.0,0.0,0.0,NaN,1.0,95.7,1.0,61551.0,0
2,130910225,7000.0,7000.0,7000.0,36 months,11.98,232.44,B,< 1 year,MORTGAGE,40000.0,Verified,home_improvement,TX,20.25,0.0,0.0,60.0,NaN,13.0,0.0,5004.0,36.0,29.0,w,0.0,60.0,0.0,131726.0,Mar-2018,0.0,0.0,0.0,64.0,3.0,89.7,0.0,132817.0,0
3,130966492,30000.0,30000.0,30000.0,36 months,21.85,1143.39,D,10+ years,OWN,57000.0,Verified,debt_consolidation,FL,27.58,0.0,1.0,68.0,NaN,11.0,0.0,29222.0,53.2,26.0,w,0.0,68.0,0.0,157566.0,Mar-2018,0.0,0.0,2.0,NaN,1.0,96.0,0.0,188780.0,0
4,130942737,21000.0,21000.0,21000.0,60 months,20.39,560.94,D,10+ years,OWN,85000.0,Source Verified,house,NY,15.76,1.0,0.0,2.0,NaN,15.0,0.0,14591.0,34.2,27.0,w,0.0,NaN,0.0,128270.0,Mar-2018,0.0,0.0,3.0,NaN,0.0,92.6,0.0,172433.0,0


In [183]:
df_copy = df.copy()

In [184]:
(df_copy.isnull().sum()/df_copy.isnull().count()).sort_values(ascending=False)[:10]

mths_since_last_record         0.807427
mths_since_recent_bc_dlq       0.757543
mths_since_last_major_derog    0.717622
mths_since_last_delinq         0.490555
emp_length                     0.069276
revol_util                     0.000796
dti                            0.000717
pct_tl_nvr_dlq                 0.000002
inq_last_6mths                 0.000002
id                             0.000000
dtype: float64

In [185]:
df_copy['mths_since_last_record'].bfill(axis='rows',inplace=True)
df_copy['mths_since_recent_bc_dlq'].bfill(axis='rows',inplace=True)
df_copy['mths_since_last_major_derog'].bfill(axis='rows',inplace=True)
df_copy['mths_since_last_delinq'].bfill(axis='rows',inplace=True)

In [186]:
df_copy.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,tot_coll_amt,tot_cur_bal,issue_d,chargeoff_within_12_mths,acc_now_delinq,mort_acc,mths_since_recent_bc_dlq,num_accts_ever_120_pd,pct_tl_nvr_dlq,pub_rec_bankruptcies,tot_hi_cred_lim,loan_status
0,130956066,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,9 years,RENT,52000.0,Source Verified,major_purchase,WA,0.58,0.0,0.0,26.0,82.0,7.0,0.0,141.0,0.5,30.0,w,0.0,74.0,0.0,150592.0,Mar-2018,0.0,0.0,4.0,64.0,0.0,96.7,0.0,191216.0,0
1,130968727,5000.0,5000.0,5000.0,36 months,11.98,166.03,B,10+ years,OWN,55000.0,Not Verified,other,GA,14.18,0.0,0.0,74.0,82.0,14.0,1.0,11449.0,33.9,24.0,w,0.0,74.0,0.0,28880.0,Mar-2018,0.0,0.0,0.0,64.0,1.0,95.7,1.0,61551.0,0
2,130910225,7000.0,7000.0,7000.0,36 months,11.98,232.44,B,< 1 year,MORTGAGE,40000.0,Verified,home_improvement,TX,20.25,0.0,0.0,60.0,117.0,13.0,0.0,5004.0,36.0,29.0,w,0.0,60.0,0.0,131726.0,Mar-2018,0.0,0.0,0.0,64.0,3.0,89.7,0.0,132817.0,0
3,130966492,30000.0,30000.0,30000.0,36 months,21.85,1143.39,D,10+ years,OWN,57000.0,Verified,debt_consolidation,FL,27.58,0.0,1.0,68.0,117.0,11.0,0.0,29222.0,53.2,26.0,w,0.0,68.0,0.0,157566.0,Mar-2018,0.0,0.0,2.0,13.0,1.0,96.0,0.0,188780.0,0
4,130942737,21000.0,21000.0,21000.0,60 months,20.39,560.94,D,10+ years,OWN,85000.0,Source Verified,house,NY,15.76,1.0,0.0,2.0,117.0,15.0,0.0,14591.0,34.2,27.0,w,0.0,40.0,0.0,128270.0,Mar-2018,0.0,0.0,3.0,13.0,0.0,92.6,0.0,172433.0,0


In [187]:
df_copy.dropna(inplace=True)

In [201]:
df_copy.drop(['id','addr_state','issue_d'],axis=1,inplace=True)

In [202]:
df_copy.select_dtypes(include='object').head()

,term,grade,emp_length,home_ownership,verification_status,purpose,initial_list_status
0,36 months,A,9 years,RENT,Source Verified,major_purchase,w
1,36 months,B,10+ years,OWN,Not Verified,other,w
2,36 months,B,< 1 year,MORTGAGE,Verified,home_improvement,w
3,36 months,D,10+ years,OWN,Verified,debt_consolidation,w
4,60 months,D,10+ years,OWN,Source Verified,house,w


In [203]:
term_values = {' 36 months': 36, ' 60 months': 60}
df_copy['term'] = df_copy.term.map(term_values)

In [204]:
LE = LabelEncoder()
df_copy['emp_length']= LE.fit_transform(df_copy['emp_length'])

In [205]:
dummies = ['grade','home_ownership','verification_status','purpose','initial_list_status']

In [206]:
data = pd.get_dummies(df_copy, columns=dummies, drop_first=True)

In [207]:
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,tot_coll_amt,tot_cur_bal,chargeoff_within_12_mths,acc_now_delinq,mort_acc,mths_since_recent_bc_dlq,num_accts_ever_120_pd,pct_tl_nvr_dlq,pub_rec_bankruptcies,tot_hi_cred_lim,loan_status,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w
0,3000.0,3000.0,3000.0,36,7.34,93.10,9,52000.0,0.58,0.0,0.0,26.0,82.0,7.0,0.0,141.0,0.5,30.0,0.0,74.0,0.0,150592.0,0.0,0.0,4.0,64.0,0.0,96.7,0.0,191216.0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,5000.0,5000.0,5000.0,36,11.98,166.03,1,55000.0,14.18,0.0,0.0,74.0,82.0,14.0,1.0,11449.0,33.9,24.0,0.0,74.0,0.0,28880.0,0.0,0.0,0.0,64.0,1.0,95.7,1.0,61551.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,7000.0,7000.0,7000.0,36,11.98,232.44,10,40000.0,20.25,0.0,0.0,60.0,117.0,13.0,0.0,5004.0,36.0,29.0,0.0,60.0,0.0,131726.0,0.0,0.0,0.0,64.0,3.0,89.7,0.0,132817.0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
3,30000.0,30000.0,30000.0,36,21.85,1143.39,1,57000.0,27.58,0.0,1.0,68.0,117.0,11.0,0.0,29222.0,53.2,26.0,0.0,68.0,0.0,157566.0,0.0,0.0,2.0,13.0,1.0,96.0,0.0,188780.0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1
4,21000.0,21000.0,21000.0,60,20.39,560.94,1,85000.0,15.76,1.0,0.0,2.0,117.0,15.0,0.0,14591.0,34.2,27.0,0.0,40.0,0.0,128270.0,0.0,0.0,3.0,13.0,0.0,92.6,0.0,172433.0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [208]:
df_target = data['loan_status']
X = data.drop('loan_status', axis = 1)

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, df_target, random_state = 10, test_size = 0.2)

print('X_train', X_train.shape)
print('y_train', y_train.shape)

print('X_test', X_test.shape)
print('y_test', y_test.shape)

X_train (385890, 55)
y_train (385890,)
X_test (96473, 55)
y_test (96473,)


In [213]:
rf_classification = RandomForestClassifier(n_estimators = 10, random_state = 10)

# use fit() to fit the model on the train set
rf_model = rf_classification.fit(X_train, y_train)

In [215]:
y_train_pred = rf_model.predict(X_train)

print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99    302009
           1       1.00      0.90      0.95     83881

    accuracy                           0.98    385890
   macro avg       0.99      0.95      0.97    385890
weighted avg       0.98      0.98      0.98    385890



In [216]:
y_test_pred = rf_model.predict(X_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.79      0.96      0.87     75242
           1       0.45      0.12      0.18     21231

    accuracy                           0.77     96473
   macro avg       0.62      0.54      0.53     96473
weighted avg       0.72      0.77      0.72     96473

